# Working with maps

In [43]:
import json
import pandas as pd
import geopandas as gpd
from IPython.display import GeoJSON

### Utilities

In [67]:
# Load and parse a GeoJSON from a remote url
def load_geojson(url):
    gdf = gpd.read_file(url)
    return json.loads(gdf.to_json())

# Create latitude and longitude columns from an object column 
def unpack_coordinates(df, column):
    return pd.concat([df.drop(column, axis=1), pd.json_normalize(df[column])], axis=1)

# Converts a pandas dataframe to a GeoJSON dict
def to_geojson(df, unpack=None, lat='latitude', lon='longitude', crs='EPSG:4326'):
    unpacked = df if unpack is None else unpack_coordinates(df, unpack)
    gdf = gpd.GeoDataFrame(unpacked, geometry=gpd.points_from_xy(unpacked[lon], unpacked[lat]), crs=crs)
    return json.loads(gdf.to_json())

## Load & plot a GeoJSON

In [75]:
# San Francisco Neighborhoods dataset
url = 'https://gist.githubusercontent.com/cdolek/d08cac2fa3f6338d84ea/raw/ebe3d2a4eda405775a860d251974e1f08cbe4f48/SanFrancisco.Neighborhoods.json'
neighborhoods = load_geojson(url)

GeoJSON(neighborhoods)

<IPython.display.GeoJSON object>

## Load a JSON and transform to GeoJSON

In [76]:
# Schools dataset from San Francisco Council Open Data
url = 'https://data.sfgov.org/resource/tpp3-epx2.json'
schools = to_geojson(pd.read_json(schools_url), unpack='location_1')

GeoJSON(schools)

<IPython.display.GeoJSON object>

## Plot a map with Leaflet

In [74]:
import ipyleaflet as L

# Choose basemap from https://ipyleaflet.readthedocs.io/en/latest/map_and_basemaps/basemaps.html#basemaps-section
# basemaps = CartoDB.DarkMatter | Strava.All | OpenStreetMap.Mapnik
# basemap = L.basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2022-01-13")
basemap = L.basemaps.OpenStreetMap.Mapnik

# Leaflet does not use GeoJSON convention for lat,lon
center = (42.5, 1.5)

L.Map(basemap=basemap, zoom=6, center=center)

Map(center=[42.5, 1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

## Find coordinates from an address with forward geocoding

In [109]:
from geopy.geocoders import Nominatim

ADDRESS = 'Casa de la Vall Andorra'

geolocator = Nominatim(user_agent="some_useragent")
address, location = geolocator.geocode(ADDRESS)
latitude, longitude = location

display(address, latitude, longitude)

map = L.Map(zoom=17, center=location)
marker = L.Marker(location=[latitude, longitude])
map.add_layer(marker);

map

'Casa de la Vall, Carrer Anna Maria Janer, Centre històric, La Comella, Andorra la Vella, AD500, Andorra'

42.5066141

1.5205063532945724

Map(center=[42.5066141, 1.5205063532945724], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…